In [16]:
import csv
import requests
from unidecode import unidecode 
import os

In [17]:
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"f810540cc3"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Opera GX";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 OPR/102.0.0.0',
    'If-Modified-Since': 'Sun, 19 Nov 2023 00:00:00 GMT'
}

In [4]:
response = requests.get(f'https://api.sofascore.com/api/v1/player/1179821', headers=headers) 
player= response.json()
player

{'player': {'name': 'Kendall Brown',
  'firstName': 'Kendall',
  'lastName': 'Brown ',
  'slug': 'brown-kendall',
  'shortName': 'K. Brown',
  'team': {'name': 'Indiana Pacers',
   'slug': 'indiana-pacers',
   'shortName': 'Pacers',
   'gender': 'M',
   'sport': {'name': 'Basketball', 'slug': 'basketball', 'id': 2},
   'tournament': {'name': 'NBA',
    'slug': 'nba',
    'category': {'name': 'USA',
     'slug': 'usa',
     'sport': {'name': 'Basketball', 'slug': 'basketball', 'id': 2},
     'id': 15,
     'flag': 'usa',
     'alpha2': 'US'},
    'uniqueTournament': {'name': 'NBA',
     'slug': 'nba',
     'primaryColorHex': '#045daf',
     'secondaryColorHex': '#cc2b31',
     'category': {'name': 'USA',
      'slug': 'usa',
      'sport': {'name': 'Basketball', 'slug': 'basketball', 'id': 2},
      'id': 15,
      'flag': 'usa',
      'alpha2': 'US'},
     'userCount': 250950,
     'id': 132,
     'country': {},
     'displayInverseHomeAwayTeams': True},
    'priority': 142,
    'id': 

In [6]:
response = requests.get('https://api.sofascore.com/api/v1/player/1179821/statistics/seasons', headers=headers)
player= response.json()
player

{'uniqueTournamentSeasons': [{'uniqueTournament': {'name': 'NBA',
    'slug': 'nba',
    'primaryColorHex': '#045daf',
    'secondaryColorHex': '#cc2b31',
    'category': {'name': 'USA',
     'slug': 'usa',
     'sport': {'name': 'Basketball', 'slug': 'basketball', 'id': 2},
     'id': 15,
     'flag': 'usa',
     'alpha2': 'US'},
    'userCount': 250950,
    'id': 132,
    'displayInverseHomeAwayTeams': True},
   'seasons': [{'name': 'NBA 22/23',
     'year': '22/23',
     'editor': False,
     'id': 45096}]},
  {'uniqueTournament': {'name': 'NBA Preseason',
    'slug': 'nba-preseason',
    'primaryColorHex': '#045daf',
    'secondaryColorHex': '#cc2b31',
    'category': {'name': 'USA',
     'slug': 'usa',
     'sport': {'name': 'Basketball', 'slug': 'basketball', 'id': 2},
     'id': 15,
     'flag': 'usa',
     'alpha2': 'US'},
    'userCount': 36004,
    'id': 2329,
    'displayInverseHomeAwayTeams': True},
   'seasons': [{'name': 'NBA Preseason 2023',
     'year': '2023',
     'ed

In [23]:
def round_percentage(value):
    if value and value != '':
        try:
            return round(float(value), 2)
        except ValueError:
            # Obsługa błędu, jeśli wartość nie jest liczbą
            return None
    else:
        return None

In [26]:
selected_keys = ["player_id", "team", "secondsPlayed","points", "twoPointsMade", "twoPointAttempts", "threePointsMade", "threePointAttempts", "freeThrowsMade", "freeThrowAttempts", "fieldGoalsMade", "fieldGoalAttempts", "rebounds", "defensiveRebounds", "offensiveRebounds", "turnovers", "blocks", "personalFouls", "assists", "steals", "fieldGoalsPercentage", "freeThrowsPercentage", "threePointsPercentage", "twoPointsPercentage"]

statistics_list = []
with open('players.csv', 'r') as players_file:
    csv_reader = csv.DictReader(players_file)
    
    for row in csv_reader:
        slug_value = row['slug']
        id_value = row['id']
        player_response = requests.get(f'https://api.sofascore.com/api/v1/player/{id_value}', headers=headers).json()
        player_details = player_response.get("player", {})
        team_details = player_details.get("team", {})
        team_name = team_details.get("name", "")
        seasons_response = requests.get(f'https://api.sofascore.com/api/v1/player/{id_value}/statistics/seasons', headers=headers).json()
        seasons_data = seasons_response.get("uniqueTournamentSeasons", [])
        season_id = None
        for season_data in seasons_data:
            seasons = season_data.get("seasons", [])
            if season_id is not None:
                break
            for season in seasons:
                if season.get("name", "") == "NBA 22/23":
                    season_id = season.get("id", "")
                    tournament = season_data.get("uniqueTournament", {})
                    tournament_id = tournament.get("id", "")
                    break
                        
        statistics_response = requests.get(
            f'https://api.sofascore.com/api/v1/player/{id_value}/unique-tournament/{tournament_id}/season/{season_id}/statistics/regularSeason',
            headers=headers
        ).json()
        statistics = statistics_response.get("statistics", {})
        selected_data = {
            "player_id": id_value, 
            "team": unidecode(team_name), 
            "secondsPlayed": statistics.get("secondsPlayed", ""),
            "points":statistics.get("points", ""), 
            "twoPointsMade":statistics.get("twoPointsMade", ""), 
            "twoPointAttempts":statistics.get("twoPointAttempts", ""), 
            "threePointsMade":statistics.get("threePointsMade", ""), 
            "threePointAttempts":statistics.get("threePointAttempts", ""), 
            "freeThrowsMade":statistics.get("freeThrowsMade", ""), 
            "freeThrowAttempts":statistics.get("freeThrowAttempts", ""), 
            "fieldGoalsMade":statistics.get("fieldGoalsMade", ""), 
            "fieldGoalAttempts":statistics.get("fieldGoalAttempts", ""), 
            "rebounds":statistics.get("rebounds", ""), 
            "defensiveRebounds":statistics.get("defensiveRebounds", ""), 
            "offensiveRebounds":statistics.get("offensiveRebounds", ""), 
            "turnovers":statistics.get("turnovers", ""), 
            "blocks":statistics.get("blocks", ""), 
            "personalFouls":statistics.get("personalFouls", ""), 
            "assists":statistics.get("assists", ""), 
            "steals":statistics.get("steals", ""), 
            "fieldGoalsPercentage":round_percentage(statistics.get("fieldGoalsPercentage", "")), 
            "freeThrowsPercentage":round_percentage(statistics.get("freeThrowsPercentage", "")), 
            "threePointsPercentage":round_percentage(statistics.get("threePointsPercentage", "")), 
            "twoPointsPercentage":round_percentage(statistics.get("twoPointsPercentage", ""))
        }
        print(selected_data)
        statistics_list.append(selected_data)
        
with open('players_statistics.csv', 'w', newline='') as statistics_file:
    csv_writer = csv.DictWriter(statistics_file, fieldnames=selected_keys)
    csv_writer.writeheader()
    csv_writer.writerows(statistics_list)
    

{'player_id': '940810', 'team': 'Philadelphia 76ers', 'secondsPlayed': 46106, 'points': 323, 'twoPointsMade': 69, 'twoPointAttempts': 117, 'threePointsMade': 48, 'threePointAttempts': 124, 'freeThrowsMade': 41, 'freeThrowAttempts': 62, 'fieldGoalsMade': 117, 'fieldGoalAttempts': 241, 'rebounds': 225, 'defensiveRebounds': 167, 'offensiveRebounds': 58, 'turnovers': 30, 'blocks': 43, 'personalFouls': 105, 'assists': 46, 'steals': 13, 'fieldGoalsPercentage': 48.55, 'freeThrowsPercentage': 66.13, 'threePointsPercentage': 38.71, 'twoPointsPercentage': 58.97}
{'player_id': '987343', 'team': 'Utah Jazz', 'secondsPlayed': 78793, 'points': 698, 'twoPointsMade': 199, 'twoPointAttempts': 410, 'threePointsMade': 58, 'threePointAttempts': 203, 'freeThrowsMade': 126, 'freeThrowAttempts': 168, 'fieldGoalsMade': 257, 'fieldGoalAttempts': 613, 'rebounds': 210, 'defensiveRebounds': 169, 'offensiveRebounds': 41, 'turnovers': 126, 'blocks': 29, 'personalFouls': 108, 'assists': 246, 'steals': 41, 'fieldGoal